In [61]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

In [62]:
quotations = pd.read_csv("../quotations231001.csv").drop(columns='sys_cooler')
print('len:', len(quotations))
quotations.head()

len: 1080


,cpu,mainboard,ram,gpu,case,power,cooler,ssd
0,18640280.0,18652877.0,16443341.0,27613427.0,21583046.0,19813847.0,21550985.0,12338942.0
1,18039569.0,18652877.0,18700841.0,18021914.0,18419840.0,19023932.0,NaN,13538642.0
2,19627808.0,18021440.0,18911780.0,18668606.0,12969350.0,21206702.0,18714860.0,17454683.0
3,19174175.0,21628184.0,18911780.0,21629063.0,NaN,19814027.0,NaN,17001050.0
4,17913710.0,18021491.0,18911780.0,18303401.0,16571885.0,18442058.0,18076145.0,16587443.0


In [63]:
cpus = quotations.cpu

In [64]:
cpu_df = pd.read_csv("../part_cpu_wb230922.csv")
mainboard_df = pd.read_csv("../part_mainboard230926.csv")
ram_df = pd.read_csv("../part_ram230921.csv")
gpu_df = pd.read_csv("../part_gpu_wb230921.csv")
case_df = pd.read_csv("../part_case230920.csv")
power_df = pd.read_csv("../part_power230920.csv")
cooler_df = pd.read_csv("../part_cooler230920.csv")
ssd_df = pd.read_csv("../part_ssd230926.csv")

In [65]:
mcb = max(cpu_df['bench_mark'])
mgb = max(gpu_df['bench_mark'])
mrc = max(ram_df['capacity'])
mncb = min(cpu_df['bench_mark'])
mngb = min(gpu_df['bench_mark'])
mnrc = min(ram_df['capacity'])
mmp = max(mainboard_df['price'])
mnmp = min(mainboard_df['price'])
mpp = max(power_df['price'])
mnpp = min(power_df['price'])
mcap = max(case_df['price'])
mncap = min(case_df['price'])
mcop = max(cooler_df['price'])
mncop = min(cooler_df['price'])
msp = max(ssd_df['price'])
mnsp = min(ssd_df['price'])

In [99]:
prices_r = dict({'cpu': [], 'gpu': [], 'ram': [], 'mainboard': [], 'power': [], 'case': [], 'cooler': [], 'ssd': [], 'total': []})
prices = dict({'cpu': [], 'gpu': [], 'ram': [], 'total': []})
pseqs = dict({'cpu': [], 'gpu': [], 'ram': [], 'mainboard': [], 'power': [], 'case': [], 'cooler': [], 'ssd': []})
# bench_r = dict({'cpu': [], 'gpu': [], 'ram': []})
bench_r = dict({'cpu': [], 'gpu': [], 'ram': [], 'total': []})
bench_rr = dict({'cpu': [], 'gpu': [], 'ram': [], 'total': []})

for row in quotations.values:
    try:
        cpu_price = cpu_df.loc[cpu_df['product_seq'] == row[0], 'price'].iloc[0]
        ram_price = ram_df.loc[ram_df['product_seq'] == row[2], 'price'].iloc[0]
        gpu_price = gpu_df.loc[gpu_df['product_seq'] == row[3], 'price'].iloc[0]
        mainboard_price = mainboard_df.loc[mainboard_df['product_seq'] == row[1], 'price']
        case_price = case_df.loc[case_df['product_seq'] == row[4], 'price']
        power_price = power_df.loc[power_df['product_seq'] == row[5], 'price']
        cooler_price = cooler_df.loc[cooler_df['product_seq'] == row[6], 'price']
        ssd_price = ssd_df.loc[ssd_df['product_seq'] == row[7], 'price']
        
        cpu_bench = cpu_df.loc[cpu_df['product_seq'] == row[0], 'bench_mark'].iloc[0]
        ram_capa = ram_df.loc[ram_df['product_seq'] == row[2], 'capacity'].iloc[0]
        gpu_bench = gpu_df.loc[gpu_df['product_seq'] == row[3], 'bench_mark'].iloc[0]

        others = []
        others.append(mainboard_df.loc[mainboard_df['product_seq'] == row[1], 'price'])
        others.append(case_df.loc[case_df['product_seq'] == row[4], 'price'])
        others.append(power_df.loc[power_df['product_seq'] == row[5], 'price'])
        others.append(cooler_df.loc[cooler_df['product_seq'] == row[6], 'price'])
        others.append(ssd_df.loc[ssd_df['product_seq'] == row[7], 'price'])
        
        total = cpu_price+gpu_price+ram_price
        for o in others:
            if not o.empty: total += o.iloc[0]
        if np.isnan(total) or np.isnan(gpu_price) or np.isnan(ram_price): continue
        prices_r['cpu'].append(cpu_price / total)
        prices_r['gpu'].append(gpu_price / total)
        prices_r['ram'].append(ram_price / total)
        # prices_r['mainboard'].append(mainboard_price / total)
        # prices_r['case'].append(case_price / total)
        # prices_r['cooler'].append(cooler_price / total)
        # prices_r['ssd'].append(ssd_price / total)
        # prices_r['power'].append(power_price / total)
        prices_r['total'].append(np.log(total / 10000000))
        
        bench_r['cpu'].append(np.log((cpu_bench - mncb) / (mcb - mncb) + 1))
        bench_r['gpu'].append(np.log((gpu_bench - mngb) / (mgb - mngb) + 1))
        bench_r['ram'].append(np.log((ram_capa - mnrc) / (mrc - mnrc) + 1))
        bench_r['total'].append(np.log(total / 10000000 + 2))
        
        clog = np.log((cpu_bench - mncb) / (mcb - mncb) + 1)
        glog = np.log((gpu_bench - mngb) / (mgb - mngb) + 1)
        rlog = np.log((ram_capa - mnrc) / (mrc - mnrc) + 1)
        tlog = np.log(total / 10000000 + 2)
        total_log = clog + glog + rlog
        bench_rr['cpu'].append(clog / total_log)
        bench_rr['gpu'].append(glog / total_log)
        bench_rr['ram'].append(rlog / total_log)
        bench_rr['total'].append(tlog / total_log)
        
        prices['cpu'].append(cpu_price)
        prices['gpu'].append(gpu_price)
        prices['ram'].append(ram_price)
        prices['mainboard'].append(mainboard_price)
        prices['case'].append(case_price)
        prices['cooler'].append(cooler_price)
        prices['ssd'].append(ssd_price)
        prices['power'].append(power_price)
        
        prices['total'].append(total)
        
        pseqs['cpu'].append(cpu_df.loc[cpu_df['product_seq'] == row[0], 'product_seq'].iloc[0])
        pseqs['gpu'].append(gpu_df.loc[gpu_df['product_seq'] == row[2], 'product_seq'].iloc[0])
        pseqs['ram'].append(ram_df.loc[ram_df['product_seq'] == row[3], 'product_seq'].iloc[0])
        pseqs['mainboard'].append(mainboard_df.loc[mainboard_df['product_seq'] == row[1], 'price'].iloc[0])
        pseqs['power'].append(power_df.loc[power_df['product_seq'] == row[5], 'price'].iloc[0])
        pseqs['case'].append(case_df.loc[case_df['product_seq'] == row[4], 'price'].iloc[0])
        pseqs['cooler'].append(cooler_df.loc[cooler_df['product_seq'] == row[6], 'price'].iloc[0])
        pseqs['ssd'].append(ssd_df.loc[ssd_df['product_seq'] == row[7], 'price'].iloc[0])
        
    except Exception as e:
        continue

In [100]:
for i in bench_rr:
    print(len(bench_rr[i]))

430
430
430
430


In [59]:
minc = min(prices['cpu'])
maxc = max(prices['cpu'])
ming = min(prices['gpu'])
maxg = max(prices['gpu'])
minr = min(prices['ram'])
maxr = max(prices['ram'])
mint = min(prices['total'])
maxt = max(prices['total'])

ValueError: min() arg is an empty sequence

In [10]:
cpn = [(x - minc) / (maxc - minc) for x in prices['cpu']]
gpn = [(x - ming) / (maxg - ming) for x in prices['gpu']]
rpn = [(x - minr) / (maxr - minr) for x in prices['ram']]
tpn = [(x - mint) / (maxt - mint) for x in prices['total']]

In [11]:
for t in prices['cpu']:
    if np.isnan(t):
        print(t)

# 프로그램 사양 분석

In [25]:
gp = pd.read_csv('../gameprograms230921.csv')
ep = pd.read_csv('../etc_programs_utf.csv')
gr = pd.read_csv(f'../gamerequirements230926.csv')
er = pd.read_csv(f'../etc_requirements231002.csv')

In [26]:
programs = pd.concat([gp, ep], ignore_index=True)
reqs = pd.concat([gr, er], ignore_index=True)

In [27]:
headers = programs.usage.unique()
avg_cpu_bench = [0] * len(headers)
avg_gpu_bench = [0] * len(headers)
avg_ram_capa = [0] * len(headers)
total_counts = []

for usage in headers:
    programs_usage = reqs[reqs['program_id'].isin(programs.loc[programs.usage == usage].index)]
    
    cpu_bench = 0
    gpu_bench = 0
    ram_capa = 0
    total = len(programs_usage)
    
    for cid in programs_usage.cpu_id:
        cpu_bench += cpu_df.loc[cid-1, 'bench_mark']
    
    for gid in programs_usage.gpu_id:
        if np.isnan(gid): continue
        gpu_bench += gpu_df.loc[gid-1, 'bench_mark']
        
    for rc in programs_usage.ram:
        if np.isnan(rc): continue
        ram_capa += rc
        
    avg_cpu_bench[np.where(headers == usage)[0][0]] = cpu_bench / total
    avg_gpu_bench[np.where(headers == usage)[0][0]] = gpu_bench / total
    avg_ram_capa[np.where(headers == usage)[0][0]] = ram_capa / total
    total_counts.append(total)

print('  CPU |   GPU |  RAM |')
print('----------------------')
for h, acb, agb, arc, tt in zip(headers, avg_cpu_bench, avg_gpu_bench, avg_ram_capa, total_counts):
    print(f'{acb:5.0f} | {agb:5.0f} | {arc:4.1f} | {h} ({tt}개)')

  CPU |   GPU |  RAM |
----------------------
 3109 |  2968 |  5.4 | 게임 (12094개)
 5852 |  1332 |  3.8 | 사무 (6개)
 4874 |     0 |  5.8 | 개발 (16개)
 6118 |  1341 | 16.0 | 영상 편집 (12개)
25142 | 10948 | 13.6 | 방송 (5개)
 7903 |  5386 |  8.4 | 이미지 편집 (15개)
 6690 |  3139 | 12.6 | 모델링 (26개)
 6461 |  1448 |  7.4 | 인코딩 (8개)
 4316 |   200 |  7.5 | 음악 작업 (16개)


# 뚜

In [56]:
bdata = np.array(pd.DataFrame(bench_rr))

In [57]:
bdata

array([], shape=(0, 9), dtype=float64)

In [42]:
k = 16
model_b = KMeans(n_clusters=k, n_init='auto')

In [43]:
model_b.fit(bdata)

C:\Users\SSAFY\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


KMeans(n_clusters=16, n_init='auto')

In [44]:
res_b = model_b.predict(bdata)

In [45]:
k = [1]

In [46]:
if not k: print(1)

In [47]:
plt.figure(figsize = (8, 8))

for i in range(k):
    plt.scatter(price_rate[res_b == i, 0], pdata[res_b == i, 3] * (maxt - mint) + mint, alpha=0.5, label=f'Group {i+1}')

plt.title('K = %d results'%k , size = 15)
plt.legend()
plt.xlabel('CPU Price Rate')
plt.ylabel('Total Quotation Price')
plt.show()

TypeError: 'list' object cannot be interpreted as an integer

<Figure size 800x800 with 0 Axes>

In [ ]:
for i in range(k):
    total_avg = np.average(np.array(prices['total'])[res_b == i])
    print(f'Group{i+1} Average Total Price: {total_avg:.0f}')

In [ ]:
for i in range(k):
    cpu_bench = 0
    for cseq in np.array(pseqs['cpu'])[res_b == i]:
        cpu_bench += cpu_df.loc[cpu_df['product_seq'] == cseq, 'bench_mark'].iloc[0]
    group_len = len(np.array(pseqs['cpu'])[res_b == i])
    print(f'Group{i+1} Average Bench: {cpu_bench/group_len:.0f}')

In [ ]:
lst = []
for i in range(k):
    cpu_price = 0
    for cseq in np.array(pseqs['cpu'])[res_b == i]:
        cpu_price += cpu_df.loc[cpu_df['product_seq'] == cseq, 'price'].iloc[0]
    group_len = len(np.array(pseqs['cpu'])[res_b == i])
    lst.append(cpu_price/group_len)
    print(f'Group{i+1} Average Price: {cpu_price/group_len:.0f}')

In [ ]:
crate = []
grate = []
rrate = []
for i in range(k):
    cpu_rate = np.average(price_rate[res_b == i, 0]) * 100
    gpu_rate = np.average(price_rate[res_b == i, 1]) * 100
    ram_rate = np.average(price_rate[res_b == i, 2]) * 100
    crate.append(cpu_rate)
    grate.append(gpu_rate)
    rrate.append(ram_rate)
    total_avg = np.average(np.array(prices['total'])[res_b == i])
    print(f'Group{i+1:2d} Average Rate: {cpu_rate:2.0f}%: {gpu_rate:2.0f}%: {ram_rate:1.0f}%: {np.sum([cpu_rate, gpu_rate, ram_rate]):2.0f}%: {total_avg:>7.0f}')

In [ ]:
grouped = np.array([crate, grate, rrate]).T

In [ ]:
grouped[5, 0]

# 모델 저장

In [ ]:
import pickle

In [ ]:
pickle.dump(model_b, open("rate_by_bench.pkl", "wb"))

In [ ]:
pickle.dump(grouped, open("rate_group.pkl", "wb"))

In [ ]:
load_model = pickle.load(open("rate_by_bench.pkl", "rb"))

In [ ]:
load_group = pickle.load(open("rate_group.pkl", "rb"))

In [ ]:
lres = load_model.predict(bdata)

In [ ]:
crate = []
grate = []
rrate = []
for i in range(k):
    cpu_rate = np.average(price_rate[lres == i, 0]) * 100
    gpu_rate = np.average(price_rate[lres == i, 1]) * 100
    ram_rate = np.average(price_rate[lres == i, 2]) * 100
    crate.append(cpu_rate)
    grate.append(gpu_rate)
    rrate.append(ram_rate)
    total_avg = np.average(np.array(prices['total'])[lres == i])
    print(f'Group{i:2d} Average Rate: {cpu_rate:2.0f}%: {gpu_rate:2.0f}%: {ram_rate:1.0f}%: {np.sum([cpu_rate, gpu_rate, ram_rate]):2.0f}%: {total_avg:>7.0f}')

In [ ]:
import time

In [ ]:
time.localtime().tm_year